#### Exploring a few things
* Obs differences
    * in schema:
        * what differs for the obs and what is the same for all buildings?
        * for the building specific ones, what is different?
            * what keys differ?
                * No keys differ
            * How do the attributes differ?
                * No attributes differ
            * Does anything differ?
                * one building has a pv nominal power of 4.0 the rest have 5.0
            * How does the raw data differ?
                * each building has its own .csv
                * Two columns differ between the buildings
                    * Equipment Electric Power [kWh] and Solar Generation [W/kW]
* Env performance
    * how long does it take the env to run through timesteps and episodes?
* Where in the obs is the building difference?
    * solar_generation might be one
    * non_shiftable_load might be one? or net_electricity_consumption or electrical_storage_soc?
    * code
        * confirm the shared obs don't differ
        * confirm which building specific obs differ or dont
            * not just any difference but uniqueness


In [2]:
import numpy as np
import time as time
from citylearn.citylearn import CityLearnEnv
import json


#### Testing obs

In [3]:
# i'm having trouble getting the actual json file to load
# schema_path = '~/projects/citylearn_2022/citylearn-2022-starter-kit/data/citylearn_challenge_2022_phase_1/schema.json'
# json_schema = json.load(schema_path)

In [14]:
schema = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(schema=schema)
schema_dict = env._CityLearnEnv__schema

shared_obs_dict = {}
building_specific_obs_dict = {}

for k, v in schema_dict['observations'].items():
    if schema_dict['observations'][k]['active'] == True:
        if schema_dict['observations'][k]['shared_in_central_agent']:
            shared_obs_dict[k] = 1
        else:
            building_specific_obs_dict[k] = 1
            
print("Number of shared obs {}".format(len(shared_obs_dict)))
print("Number of building specific obs {}".format(len(building_specific_obs_dict)))

# testing the obs numbers match up
assert len(env.reset()[0]) == len(shared_obs_dict) + len(building_specific_obs_dict)
print(shared_obs_dict)
print(building_specific_obs_dict)

Number of shared obs 20
Number of building specific obs 8
{'month': 1, 'day_type': 1, 'hour': 1, 'outdoor_dry_bulb_temperature': 1, 'outdoor_dry_bulb_temperature_predicted_6h': 1, 'outdoor_dry_bulb_temperature_predicted_12h': 1, 'outdoor_dry_bulb_temperature_predicted_24h': 1, 'outdoor_relative_humidity': 1, 'outdoor_relative_humidity_predicted_6h': 1, 'outdoor_relative_humidity_predicted_12h': 1, 'outdoor_relative_humidity_predicted_24h': 1, 'diffuse_solar_irradiance': 1, 'diffuse_solar_irradiance_predicted_6h': 1, 'diffuse_solar_irradiance_predicted_12h': 1, 'diffuse_solar_irradiance_predicted_24h': 1, 'direct_solar_irradiance': 1, 'direct_solar_irradiance_predicted_6h': 1, 'direct_solar_irradiance_predicted_12h': 1, 'direct_solar_irradiance_predicted_24h': 1, 'carbon_intensity': 1}
{'non_shiftable_load': 1, 'solar_generation': 1, 'electrical_storage_soc': 1, 'net_electricity_consumption': 1, 'electricity_pricing': 1, 'electricity_pricing_predicted_6h': 1, 'electricity_pricing_predic

In [10]:
building_name_dict = {}
building_feature_dict = {}
attribute_dict = {}


for k, v in schema_dict['buildings'].items():
    building_name_dict[k] = k
    
for k, v in building_name_dict.items():
    # print(schema_dict['buildings'][k])
    for k2, v2 in schema_dict['buildings'][k].items():
        # STOPPED HERE, KEYS ARE SAME BUT NEED NEW DICT OF VALUES TO SEE WHAT IS DIFFERENT
        if k2 in building_feature_dict:
            building_feature_dict[k2] += 1
            if v2 in building_feature_dict[k2+"_values"]:
                building_feature_dict[k2+"_values"][v2] += 1
            else:
                building_feature_dict[k2+"_values"][v2] = 1
        else:
            building_feature_dict[k2] = 1
            #STOPPED HERE EMPTY LIST STUFF
            building_feature_dict[k2+"_values"] = {v2: 1}
        # see attributes that are different
    # go through each building, and get the different things by the keys
    # schema_dict['buildings']
    # if k not in attributes_dict:
    #     attributes_dict[k] = {}
    # else:
#print(building_name_dict, building_feature_dict, attribute_dict)
print( building_feature_dict )

TypeError: unhashable type: 'list'

In [13]:
schema_dict['buildings'][k][k2]

[]

In [12]:
k2

'inactive_observations'

In [6]:
schema_dict['buildings']

{'Building_1': {'include': True,
  'energy_simulation': 'Building_1.csv',
  'weather': 'weather.csv',
  'carbon_intensity': 'carbon_intensity.csv',
  'pricing': 'pricing.csv',
  'inactive_observations': [],
  'inactive_actions': [],
  'electrical_storage': {'type': 'citylearn.energy_model.Battery',
   'autosize': False,
   'attributes': {'capacity': 6.4,
    'efficiency': 0.9,
    'capacity_loss_coefficient': 1e-05,
    'loss_coefficient': 0.0,
    'nominal_power': 5.0,
    'seconds_per_time_step': 3600}},
  'pv': {'type': 'citylearn.energy_model.PV',
   'autosize': False,
   'attributes': {'nominal_power': 4.0, 'seconds_per_time_step': 3600}}},
 'Building_2': {'include': True,
  'energy_simulation': 'Building_2.csv',
  'weather': 'weather.csv',
  'carbon_intensity': 'carbon_intensity.csv',
  'pricing': 'pricing.csv',
  'inactive_observations': [],
  'inactive_actions': [],
  'electrical_storage': {'type': 'citylearn.energy_model.Battery',
   'autosize': False,
   'attributes': {'capac

#### env Performance

In [48]:
import time
schema = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(schema=schema)


num_episodes = 10
num_timesteps = 0
sample_action = [[.1]]*5
for e in range(num_episodes):
    env.reset()
    time_start = time.time()
    done = False
    while not done:
        num_timesteps += 1
        _, _, done, _ = env.step(sample_action)
        
    run_time = time.time() - time_start
    print("Episode {} finished in {:.1f} seconds. Timesteps per second {:.1f} ".format(e, run_time, num_timesteps/run_time))

Episode 0 finished in 596.3 seconds. Timesteps per second 14.7 
Episode 1 finished in 532.3 seconds. Timesteps per second 32.9 
Episode 2 finished in 526.8 seconds. Timesteps per second 49.9 
Episode 3 finished in 528.1 seconds. Timesteps per second 66.3 
Episode 4 finished in 529.8 seconds. Timesteps per second 82.7 
Episode 5 finished in 528.3 seconds. Timesteps per second 99.5 
Episode 6 finished in 528.0 seconds. Timesteps per second 116.1 
Episode 7 finished in 528.4 seconds. Timesteps per second 132.6 
Episode 8 finished in 529.8 seconds. Timesteps per second 148.8 
Episode 9 finished in 528.4 seconds. Timesteps per second 165.8 


In [55]:
8200/530

15.471698113207546

#### Confirming the obs are the same
* Is a picture on the discord that has the order of them
* Each obs is a list of 5 lists
    * zip the lists
    * confirm the lengths
    * if the obs are not the same then store in a dict
        * the obs index, the timestep, the number of different entries, the entries themselves
        * maybe print the index as well

In [49]:
# env.reset()

In [50]:
def tell_obs_differences(obs, oc_dict, ts):
    num_obs = len(obs)
    # check obs length
    obs_length = len(obs[0])
    for i in range(1, num_obs):
        if len(obs[i]) != obs_length:
            print("WTF the obs are different lengths", ts, len(obs[i]), obs_length, i)
            print(obs)
            
    # check if any obs are different
    # iter over obs index
    for obs_index in range(0, obs_length):
        # iter over the buildings
        check_dict = {}
        
        for obs_building in range(0, num_obs):
            current_obs = obs[obs_building][obs_index]
            if current_obs in check_dict:
                check_dict[current_obs] += 1
            else:
                check_dict[current_obs] = 1
        
        # should only be one entry if all the same
        if len(check_dict.keys()) > 1:
            ts_dict = {
                'num_differences':len(check_dict.keys()), 
                    'actual_obs':check_dict
            }
            if obs_index not in oc_dict:
                oc_dict[obs_index] = {}
                oc_dict[obs_index] = {'ts_different':1}
            else:
                oc_dict[obs_index]['ts_different'] += 1
            oc_dict[obs_index][ts] = ts_dict
                
    return oc_dict

In [51]:
import time
schema = 'citylearn_challenge_2022_phase_1'
env = CityLearnEnv(schema=schema)

obs_check_dict = {}

num_episodes = 1
num_timesteps = 0
sample_action = [[.1]]*5
for e in range(num_episodes):
    env.reset()
    time_start = time.time()
    done = False
    while not done:
        num_timesteps += 1
        obs, _, done, _ = env.step(sample_action)
        obs_check_dict = tell_obs_differences(obs, obs_check_dict, num_timesteps)
    run_time = time.time() - time_start
    print("Episode {} finished in {:.1f} seconds. Timesteps per second {:.1f} ".format(e, run_time, num_timesteps/run_time))

Episode 0 finished in 511.0 seconds. Timesteps per second 17.1 


In [52]:
for k, v in obs_check_dict.items():
    print(k, obs_check_dict[k]['ts_different'])

20 8759
23 8759
21 4366


In [53]:
import pickle

with open('obs_check_dict.pkl', 'wb') as handle:
    pickle.dump(obs_check_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### How fast is cartpole?

In [60]:
import gym

cp_env = gym.make('CartPole-v0')

num_episodes = 10
for e in range(num_episodes):
    num_timesteps = 0
    cp_env.reset()
    time_start = time.time()
    done = False
    while not done:
        num_timesteps += 1
        _, _, done, _ = cp_env.step(cp_env.action_space.sample())
        #obs_check_dict = tell_obs_differences(obs, obs_check_dict, num_timesteps)
    run_time = time.time() - time_start
    print("Episode {} finished in {:.1f} seconds. Timesteps per second {:.1f} ".format(e, run_time, num_timesteps/run_time))

Episode 0 finished in 0.0 seconds. Timesteps per second 31561.5 
Episode 1 finished in 0.0 seconds. Timesteps per second 93305.5 
Episode 2 finished in 0.0 seconds. Timesteps per second 97400.4 
Episode 3 finished in 0.0 seconds. Timesteps per second 100071.6 
Episode 4 finished in 0.0 seconds. Timesteps per second 64927.3 
Episode 5 finished in 0.0 seconds. Timesteps per second 78299.3 
Episode 6 finished in 0.0 seconds. Timesteps per second 97542.0 
Episode 7 finished in 0.0 seconds. Timesteps per second 96866.1 
Episode 8 finished in 0.0 seconds. Timesteps per second 101338.9 
Episode 9 finished in 0.0 seconds. Timesteps per second 95158.7 


1

In [62]:
obs_list = env.reset()

In [71]:
shared_obs_list = obs_list[0][3:20] + obs_list[0][22:23] + obs_list[0][24:]
len(shared_obs_list)

22

In [67]:
obs_list[0][22]

0.0

In [72]:
building_obs_list = []
for i in range(len(obs_list)):
    building_obs_list = building_obs_list + obs_list[i][20:22] + obs_list[i][23:24]
len(building_obs_list)

15

In [74]:
len(env.observation_space)

5

In [75]:
env.observation_space[0].shape[0]

28

In [78]:
a = [1,4,1,23]
b = np.array(a, dtype='float32')
b.shape

(4,)

In [79]:
b

array([ 1.,  4.,  1., 23.], dtype=float32)

In [81]:
env.observation_space[0].low[0]

0.0

In [87]:
 len(env.observation_space[0].low[3:20].tolist() + env.observation_space[0].low[22:23].tolist() + env.observation_space[0].low[24:].tolist())

22

In [90]:
a + a

[1, 4, 1, 23, 1, 4, 1, 23]

In [93]:
a = [1,4,1,23]
a.extend(a*5)
print(a)
print(len(a))

[1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23]
24


In [92]:
a

[1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23, 1, 4, 1, 23]

In [94]:
[-1]*6

[-1, -1, -1, -1, -1, -1]

In [100]:
-1 * np.array(a)

array([ 10,  40,  10, 230,  10,  40,  10, 230,  10,  40,  10, 230,  10,
        40,  10, 230,  10,  40,  10, 230,  10,  40,  10, 230])